In [37]:
import pandas as pd
import numpy as np
 
# Define the datasets and ablation types
datasets = ['adult', 'taxi', 'wine']
ablation_types = ['full', 'no_variance_filter', 'no_dim_reduction', 'no_iterative_filter']
 
# Create results dictionary
results = {}
 
base_path = "./results"
 
for dataset in datasets:
    print(dataset)
    for ablation in ablation_types:
        print(ablation)
 
        key = f"{dataset.capitalize()}_{ablation}"
        
        try:
            # Read Silhouette and DBI
            cluster_quality_path = f"{base_path}/{dataset}/{ablation}/cluster_quality/overall_cluster_quality_scores.csv"
            cluster_df = pd.read_csv(cluster_quality_path)
            silhouette_avg = cluster_df['Silhouette_Score'].values[0]
            dbi_avg = cluster_df['Davies_Bouldin_Index'].values[0]
            
            # Read min and max silhouette from per_cluster_silhouette_metrics.csv
            per_cluster_path = f"{base_path}/{dataset}/{ablation}/cluster_quality/per_cluster_silhouette_metrics.csv"
            try:
                per_cluster_df = pd.read_csv(per_cluster_path)
                n_clusters = len(per_cluster_df)
                silhouette_min = per_cluster_df['silhouette_min'].min()
                silhouette_max = per_cluster_df['silhouette_max'].max()
            except:
                n_clusters = None
                silhouette_min = None
                silhouette_max = None
            
            # Read representative quality detailed for CV metrics
            rep_quality_path = f"{base_path}/{dataset}/{ablation}/representative_quality/representative_quality_detailed.csv"
            try:
                rep_df = pd.read_csv(rep_quality_path)
                
                # Count selected and non-selected metrics (average across all clusters)
                n_selected = len(rep_df[rep_df['Type'] == 'Selected'].groupby('Cluster').first())
                n_non_selected = len(rep_df[rep_df['Type'] == 'Non-Selected'].groupby('Cluster').first())
                
                # Average these counts across clusters
                selected_counts = rep_df[rep_df['Type'] == 'Selected'].groupby('Cluster').size()
                non_selected_counts = rep_df[rep_df['Type'] == 'Non-Selected'].groupby('Cluster').size()
                
                n_selected_avg = selected_counts.mean() if len(selected_counts) > 0 else 0
                n_non_selected_avg = non_selected_counts.mean() if len(non_selected_counts) > 0 else 0
                
                # Get CV statistics (all CV values across all clusters)
                cv_values = rep_df['CV'].values
                print(cv_values)
                cv_avg = cv_values.mean()
                cv_min = cv_values.min()
                cv_max = cv_values.max()
                
                n_selected_str = f"{n_selected_avg:.1f}"
                n_non_selected_str = f"{n_non_selected_avg:.1f}"
            except Exception as e:
                print(f"Could not load representative quality for {key}: {e}")
                n_selected_str = "N/A"
                n_non_selected_str = "N/A"
                cv_avg = None
                cv_min = None
                cv_max = None
            
            # Read QSE - try multiple possible filenames
            qse_path = None
            for filename in ['qse_metrics.csv', 'qse_best_per_cluster.csv', 'qse_all_rules.csv']:
                test_path = f"{base_path}/{dataset}/{ablation}/explanation_quality/{filename}"
                try:
                    qse_df = pd.read_csv(test_path)
                    qse_path = test_path
                    break
                except:
                    continue
            
            if qse_path:
                qse_df = pd.read_csv(qse_path)
                if 'qse' in qse_df.columns:
                    qse_avg = qse_df['qse'].mean()
                elif 'QSE' in qse_df.columns:
                    qse_avg = qse_df['QSE'].mean()
                else:
                    # Try the first numeric column
                    numeric_cols = qse_df.select_dtypes(include=[np.number]).columns
                    qse_avg = qse_df[numeric_cols[0]].mean() if len(numeric_cols) > 0 else None
            else:
                qse_avg = None
            
            # Read F1 Score from predictive quality metrics
            f1_path = f"{base_path}/{dataset}/{ablation}/predictive_quality/predictive_quality_metrics.csv"
            try:
                f1_df = pd.read_csv(f1_path)
                if 'F1 Score' in f1_df.columns:
                    f1_scores = f1_df['F1 Score'].values
                    f1_avg = f1_scores.mean()
                    # Check if all zeros
                    if np.all(f1_scores == 0):
                        f1_str = "0.0000"
                    else:
                        f1_str = f"{f1_avg:.4f}"
                else:
                    f1_str = "N/A"
            except Exception as e:
                print(f"Could not load F1 scores for {key}: {e}")
                f1_str = "N/A"
            
            results[key] = {
                'Dataset': dataset.capitalize(),
                'Ablation': ablation.replace('_', ' ').title(),
                'N_Clusters': n_clusters if n_clusters is not None else "N/A",
                'SiS_avg': f"{silhouette_avg:.4f}",
                'SiS_min': f"{silhouette_min:.4f}" if silhouette_min is not None else "N/A",
                'SiS_max': f"{silhouette_max:.4f}" if silhouette_max is not None else "N/A",
                'DBI_avg': f"{dbi_avg:.4f}",
                'CV_avg': f"{cv_avg:.4f}" if cv_avg is not None else "N/A",
                'CV_min': f"{cv_min:.4f}" if cv_min is not None else "N/A",
                'CV_max': f"{cv_max:.4f}" if cv_max is not None else "N/A",
                'F1_avg': f1_str,
                'QSE_avg': f"{qse_avg:.4f}" if qse_avg is not None else "N/A"
            }
        except Exception as e:
            print(f"Could not load {key}: {e}")
 
# Create dataframe from results and sort by Dataset then Ablation
results_df = pd.DataFrame(list(results.values()))
results_df = results_df.sort_values(by=['Dataset', 'Ablation']).reset_index(drop=True)

# Display the table
print("\nResults Table for Paper (sorted by Dataset):")
print("=" * 160)
print(results_df.to_string(index=False))
print("=" * 160)

# Create a version with bolded winners for CSV
csv_df = results_df.copy()

# Define which columns are "higher is better" and "lower is better"
higher_is_better = ['SiS_avg', 'SiS_min', 'SiS_max', 'F1_avg', 'QSE_avg']  # These want max
lower_is_better = ['DBI_avg', 'CV_avg', 'CV_min', 'CV_max']  # These want min

# Bold the winners in each column, per dataset
for col in csv_df.columns:
    if col in ['Dataset', 'Ablation', 'N_Clusters']:
        continue  # Skip non-numeric columns
    
    try:
        # For each dataset, bold the best value
        for dataset in csv_df['Dataset'].unique():
            dataset_indices = csv_df[csv_df['Dataset'] == dataset].index
            
            # Convert strings to floats for comparison within this dataset
            numeric_values = pd.to_numeric(csv_df.loc[dataset_indices, col], errors='coerce')
            
            if numeric_values.isna().all():
                continue  # Skip if all N/A
            
            if col in higher_is_better:
                # Bold the maximum value(s) in this dataset
                max_val = numeric_values.max()
                if pd.notna(max_val):
                    idx = numeric_values[numeric_values == max_val].index[0]
                    current_val = csv_df.loc[idx, col]
                    if not current_val.startswith('\\textbf'):
                        csv_df.loc[idx, col] = f"\\textbf{{{current_val}}}"
            
            elif col in lower_is_better:
                # Bold the minimum value(s) in this dataset
                min_val = numeric_values.min()
                if pd.notna(min_val):
                    idx = numeric_values[numeric_values == min_val].index[0]
                    current_val = csv_df.loc[idx, col]
                    if not current_val.startswith('\\textbf'):
                        csv_df.loc[idx, col] = f"\\textbf{{{current_val}}}"
    except:
        continue

# Save to CSV
csv_df.to_csv("./results_table_for_paper.csv", index=False)
print("\nTable saved to: ./results_table_for_paper.csv")


adult
full
[0.01754673 0.01369074 0.00535752 0.05187119 0.01548012 0.01518664
 0.00420299 0.07376359 0.00323346 0.0038797  0.25647832 0.01530798
 0.02575462 0.01483029 0.02575462 0.00456818 0.01483029 0.01047634
 0.00813843 0.01141076 0.23689382 0.17584329 0.20720149 0.53254453
 0.01377968 0.00593971 0.00412153 0.0033834  0.01506886 0.01243728
 0.087566   2.9104275  0.14658098 0.6658236  0.86546324 0.6658236
 0.02016986 0.60922477 0.86546324 0.1276725  0.08433535 0.01559356
 2.9104275  0.17629265 0.03073722 0.03762532 2.9104275  0.22228958
 0.0298657  0.00983232 0.08143489 0.0061728  0.00436536 0.02698206
 0.06661522 0.66560383 0.72730898 0.66560383 0.00605015 0.65878794
 0.72730898 0.06403447 0.00513191 0.03546236 0.00532125 0.13423293
 0.01371281 0.0049488  0.4879261  0.07524042 0.01665697 0.00623978
 0.01586155 0.0041296  0.08616257 0.00437175 0.08143127 0.01538126
 0.04982629 0.06976155 0.04982629 0.00382918 0.06175275 0.06976155
 0.00689916 0.01759296 0.06561791 0.01158033 0.00849